In [ ]:
import os

output_path=os.path.join(os.getcwd(), "data") #Folder name 'data' in your work directory

In [ ]:
import requests
import os
import urllib.request
from tqdm import tqdm
from PIL import Image
import io



def get_all_breeds():
    response = requests.get("https://dog.ceo/api/breeds/list/all")
    data = response.json()
    return data["message"]


def get_images_for_breeds(breeds):
    images = {}

    for breed, sub_breeds in breeds.items():
        images[breed] = get_images_by_breed(breed)

    return images


def get_images_by_breed(breed):
    response = requests.get(f"https://dog.ceo/api/breed/{breed}/images")
    data = response.json()
    return data["message"]


def download_images(images, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for breed, urls in tqdm(images.items(), desc="Downloading images"):
        breed_dir = os.path.join(target_dir, breed)
        if not os.path.exists(breed_dir):
            os.makedirs(breed_dir)

        for i, url in enumerate(urls):
            image_path = os.path.join(breed_dir, f"{breed}_{i}.jpg")
            try:
                # Retrieve the image from the URL
                response = requests.get(url)
                response.raise_for_status()
                
                # Attempt to open the image to check if it's readable
                img = Image.open(io.BytesIO(response.content))
                
                # Save the image if it's valid
                with open(image_path, 'wb') as f:
                    f.write(response.content)
            except Exception as e:
                print(f"Error downloading {url}: {e}")
                

In [ ]:
breeds = get_all_breeds()
images = get_images_for_breeds(breeds)
download_images(images, target_dir=output_path)